In [31]:
from decouple import config
from pylabnet.utils.logging.logger import LogClient
import paramiko
from decouple import config
import time
import numpy as np


In [32]:
LOCALHOST_PW = config('LOCALHOST_PW')


In [33]:
# Instantiate

loghost = '140.247.189.94'
logport = 28381
logger = LogClient(
    host=loghost,
    port=logport,
    module_tag='Test'
)

In [34]:
from pylabnet.utils.helper_methods import show_console, hide_console, load_config
hosts = load_config('ssh_config', logger=logger)['hosts']

In [39]:
for host in hosts:

        # Initiate SSH connection
        hostname = host['hostname']
        host_ip = host['ip']

        logger.info(f"Starting SSH connection to {hostname}@{host_ip}")

        ssh = paramiko.SSHClient()
        ssh.load_system_host_keys()

        try:
            ssh.connect(host_ip, username=hostname, password=LOCALHOST_PW)
        except TimeoutError:
            logger.warn(f"Failed to setup SSH connection to {hostname}@{host_ip}")

        logger.warn(f"Connected via SSH to {hostname}@{host_ip}")

        python_path = host['python_path']
        script_path = host['script_path']
        venv_path =  host['venv_path']
        servers = host['servers']

        for server in servers:

            servername = server['servername']
            server_port = np.random.randint(1, 9999)

            # Activate virtual env
            ssh.exec_command(venv_path)
        
    
            cmd = '"{}" "{}" --logip {} --logport {} --serverport {} --server {} --debug {}'.format(
                        python_path,
                        script_path,
                        loghost,
                        logport,
                        server_port,
                        servername,
                        False
                    )

            ssh.exec_command(cmd)   

        
        ssh.close()

In [ ]:
C:\Users\Dolores\pylabnet\venv\dev\Scripts activate


In [17]:
stdin, stdout, stderr = ssh.exec_command("ls -l ~")
time.sleep(2)    # Previously, I had to sleep for some time.
stdout_.channel.recv_exit_status()
lines = stdout_.readlines()
for line in lines:
    print line

In [23]:

ssh.close()

In [22]:
stdout

<paramiko.ChannelFile from <paramiko.Channel 2 (closed) -> <paramiko.Transport at 0x26e6bda0 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>>>

# Playback Sawthooth Waveform

In [12]:
# AOM Switch 
import pylabnet.hardware.ni_daqs.nidaqmx_card as nidaqmx
import pylabnet.network.client_server.nidaqmx_card as nidaqmx_card_server
import numpy as np
import time

In [13]:
aom_daq = nidaqmx_card_server.Client(
    host='140.247.189.82',
    port=24195
)

In [28]:
amplitude = 1 # in V
offset = 0 # in V
num_points = 50
frequency = 0.02 # in Hz

In [29]:
period = 1/frequency
scanning_range_up = np.linspace(offset, offset+amplitude, num_points)
scanning_range_down = scanning_range_up[::-1]

In [30]:

for voltage in np.concatenate((scanning_range_up,scanning_range_down)):
    aom_daq.set_ao_voltage('ao4', voltage)
    time.sleep(period / (2 * num_points))


In [27]:
aom_daq.set_ao_voltage('ao4', 0)
